# I3D Preprocessing

In [ ]:
import cv2
import numpy as np
import os
import json
import psutil
import gc
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
import pandas as pd

In [ ]:
# Paths
VIDEO_PATH = "./DataSet/videos"  # Path to raw videos
SAVE_PATH = "./DataSet/I3D_Processed"  # Path to save processed .npy files
METADATA_PATH = "./DataSet/WLASL_v0.3.json"  # Path to metadata
TOP_100_PATH = "./DataSet/gloss_counts_top_100.csv"  # Path to top 100 glosses

# Processing Parameters
NUM_FRAMES = 64  # Fixed number of frames for each video
FRAME_HEIGHT = 224  # Height for resizing frames
FRAME_WIDTH = 224  # Width for resizing frames
BATCH_SIZE = 5  # Batch size for loading frames to manage memory

os.makedirs(SAVE_PATH, exist_ok=True)


In [ ]:
# Load WLASL metadata
with open(METADATA_PATH, 'r') as f:
    metadata = json.load(f)

# Map video IDs to gloss labels
video_label_map = {}
for entry in metadata:
    label = entry['gloss']
    for instance in entry['instances']:
        video_id = int(instance['video_id'])
        video_label_map[video_id] = label

# Load top 100 glosses
df = pd.read_csv(TOP_100_PATH)
top_100_classes = df['Gloss'].tolist()

In [ ]:
def preprocess_video(video_path, num_frames=NUM_FRAMES, target_size=(FRAME_HEIGHT, FRAME_WIDTH)):
    """Extract, resize, and pad/truncate frames from a video."""
    try:
        cap = cv2.VideoCapture(video_path)
        frames = []
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if frame_count == 0:
            print(f"Skipping video (no frames found): {video_path}")
            return None

        # Sample frames uniformly
        for i in range(min(num_frames, frame_count)):
            cap.set(cv2.CAP_PROP_POS_FRAMES, int(i * frame_count / num_frames))
            success, frame = cap.read()
            if success:
                frame = cv2.resize(frame, target_size)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = frame.astype(np.float32) / 255.0  # Normalize to [0, 1]
                frames.append(frame)

        cap.release()

        # Stack frames and pad if necessary
        frames = np.stack(frames, axis=0)
        if len(frames) < num_frames:
            padding = np.zeros((num_frames - len(frames), *frames.shape[1:]))
            frames = np.vstack([frames, padding])

        return frames

    except Exception as e:
        print(f"Error processing video {video_path}: {e}")
        return None

In [ ]:
def process_and_save_video(video_file):
    """Process a video and save the frames as .npy."""
    video_path = os.path.join(VIDEO_PATH, video_file)
    video_id = int(video_file.split('.')[0])

    if video_id in video_label_map:
        label = video_label_map[video_id]
    else:
        print(f"Warning: No label found for video {video_file}")
        return

    frames = preprocess_video(video_path)
    if frames is not None:
        action_save_dir = os.path.join(SAVE_PATH, label)
        os.makedirs(action_save_dir, exist_ok=True)
        np.save(os.path.join(action_save_dir, f"{video_id}_frames.npy"), frames)
        print(f"Saved {video_file} as {label}_frames.npy")


In [ ]:
video_files = [f for f in os.listdir(VIDEO_PATH) if f.endswith('.mp4')]
with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(process_and_save_video, video_files), total=len(video_files), desc="Processing Videos"))

In [ ]:
# Function to check memory usage
def check_memory():
    memory = psutil.virtual_memory()
    print(f"Memory usage: {memory.percent}% of {memory.total / (1024 ** 3):.2f} GB")

# Initialize variables
sequences = []
labels = []

# Load data for top 100 glosses
for action in tqdm(top_100_classes, desc="Loading data for top 100 classes"):
    action_dir = os.path.join(SAVE_PATH, action)

    if os.path.exists(action_dir):
        batch_sequences = []
        batch_labels = []

        for video_file in os.listdir(action_dir):
            if video_file.endswith('_frames.npy'):
                frames = np.load(os.path.join(action_dir, video_file))
                batch_sequences.append(frames)
                batch_labels.append(action)

                # Process in batches
                if len(batch_sequences) == BATCH_SIZE:
                    sequences.extend(batch_sequences)
                    labels.extend(batch_labels)
                    batch_sequences = []
                    batch_labels = []
                    check_memory()
                    gc.collect()

        # Process remaining videos
        if batch_sequences:
            sequences.extend(batch_sequences)
            labels.extend(batch_labels)
            check_memory()
            gc.collect()
    else:
        print(f"Missing directory for class: {action}")


In [ ]:
X = np.array(sequences)
y = [top_100_classes.index(label) for label in labels]
y = to_categorical(y, num_classes=len(top_100_classes))

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")


In [ ]:
np.save('./Dataset/final_Dataset/X_i3d_100.npy', X)
np.save('./Dataset/final_Dataset/y_i3d_100.npy', y)
